In [20]:
import torch
from torch import nn
from torch.functional import F

In [2]:

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2910],
        [-0.3978]], grad_fn=<AddmmBackward0>)

In [3]:
# 参数访问
print(net)
# 可以发现0和2是全连接层，可以访问其w和b
print(net[0].state_dict())
print(net[2].state_dict())

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
OrderedDict([('weight', tensor([[ 0.0191, -0.3707,  0.3710,  0.0186],
        [-0.1848,  0.2448,  0.1824, -0.0873],
        [ 0.4983, -0.4516, -0.0968,  0.2754],
        [ 0.3608,  0.0926, -0.0296,  0.2628],
        [ 0.1947,  0.0518,  0.1387,  0.3038],
        [-0.2839, -0.2572,  0.2698,  0.4630],
        [-0.0135,  0.1464,  0.3729,  0.4516],
        [-0.4159,  0.2371, -0.0935, -0.3971]])), ('bias', tensor([ 0.2260,  0.2860, -0.2900, -0.0461,  0.3224, -0.0508, -0.1791, -0.2862]))])
OrderedDict([('weight', tensor([[ 0.0888, -0.2260,  0.2709, -0.2369,  0.2085, -0.1214, -0.1154,  0.1859]])), ('bias', tensor([-0.2957]))])


In [4]:
# 获取参数数值，参数是复合的对象，包含值、梯度和额外信息
# 获取b
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
#获取w
print(type(net[2].weight))
print(net[2].weight)
print(net[2].weight.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2957], requires_grad=True)
tensor([-0.2957])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([[ 0.0888, -0.2260,  0.2709, -0.2369,  0.2085, -0.1214, -0.1154,  0.1859]],
       requires_grad=True)
tensor([[ 0.0888, -0.2260,  0.2709, -0.2369,  0.2085, -0.1214, -0.1154,  0.1859]])


In [5]:
# 访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([-0.2957])

In [7]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这⾥嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3357],
        [0.3357]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
# 访问这个嵌套块的某一层参数
print(rgnet[0][1][0])
print(rgnet[0][1][0].bias.data)

Linear(in_features=4, out_features=8, bias=True)
tensor([-0.0353, -0.1905,  0.4204, -0.1027,  0.1336,  0.3835, -0.4901, -0.4334])


In [12]:
# 初始化网络模型参数
# 深度学习框架提供默认随机初始化,可以自定义网络模型初始化参数
"""
如下将Liner的权重参数初始化为标准差为0.01的⾼斯随机变量，且将偏置参数设置为0
"""
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0090, -0.0109,  0.0089,  0.0090]), tensor(0.))

In [13]:
"""
初始化指定常量
"""
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [14]:
# 不同网络层使用不同初始化方法
"""
使⽤Xavier初始化⽅法初始化第⼀个神经⽹络层，然后将第三个神经⽹络层初始化为常量值42
"""
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.2479,  0.2262,  0.0314, -0.2129])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
# 如果需要完全自定义，可以通过赋值的方式
def my_init(m):
    if type(m)==nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.zeros_(m.weight)
        m.weight.data+=233
net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[233., 233., 233., 233.],
        [233., 233., 233., 233.],
        [233., 233., 233., 233.],
        [233., 233., 233., 233.],
        [233., 233., 233., 233.],
        [233., 233., 233., 233.],
        [233., 233., 233., 233.],
        [233., 233., 233., 233.]], requires_grad=True)

In [17]:
# 参数可以随时，在外部赋值进行改变
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([ 42., 234., 234., 234.])

In [18]:
# 参数绑定
"""
如果需要在多个层间共享参数，可以定义⼀个稠密层，然后使⽤它的参数来设置另⼀个层的参数
"""
# 定义一个共享层shared
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),shared, nn.ReLU(),shared, nn.ReLU(),
nn.Linear(8, 1))
net(X)

tensor([[-0.3033],
        [-0.2975]], grad_fn=<AddmmBackward0>)

In [19]:
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同⼀个对象，⽽不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])
"""
共享层对反向传播的影响：
由于模型参数包含梯度，因此在反向传播期间第⼆个隐藏层（即第三个神经⽹络层）和第三个隐藏层（即第五个神经⽹络层）的梯度会加在⼀起
"""

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [21]:
# 自定义网络层设置参数
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # 自定义一个weight和bias参数
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.9957,  0.2960, -0.1630],
        [ 0.8267,  0.9739,  1.0168],
        [-0.0880,  0.0558,  0.2509],
        [ 0.3167,  1.1399, -1.2147],
        [ 0.7683, -1.0545, -0.3660]], requires_grad=True)